# AAO GALAH RESEARCH JAN-APR 2016

### Missy McIntosh, Gayandhi de Silva, Jeffrey Simpson
http://www.univie.ac.at/webda/cgi-bin/frame_data_list.cgi?ascc111+ad2k+ad2000.coo

#### Check if there are any known open clusters in GALAH
#### This is a very slow python script....

# Table-of-Contents


[Load 2Mass database](#Load-2Mass-Database) 

[Select Clusters](#Select-Clusters) 

[Webda Data Retrieval](#Webda-Data-Retrieval) 

[Database Search](#Database-Search) 

[Filtering](#Filtering) 

[Looping](#Looping) 


In [1]:
%matplotlib inline
%load_ext Cython
import matplotlib.pyplot as plt
import numpy as np
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.coordinates import FK4
import time
from tqdm import *
import pandas as pd
import codecs
import numpy as np
from lxml import html
from lxml import etree
import requests
import csv
from numpy import genfromtxt
import cython
import pyximport
pyximport.install(setup_args={"include_dirs":np.get_include()},
                  reload_support=True)
import glob
import pylab as py
from scipy import optimize
from scipy.stats import chisquare
from scipy.stats import norm




/Users/mmcintosh/anaconda/lib/python3.4/site-packages/IPython/kernel/__init__.py:13: ShimWarning: The `IPython.kernel` package has been deprecated. You should import from ipykernel or jupyter_client instead.
  "You should import from ipykernel or jupyter_client instead.", ShimWarning)


In [100]:
# Equation for Gaussian
def gauss(x, a, b, c):
    return a * py.exp(-(x - b)**2.0 / (2 * c**2))

def coordseparation(a, b):
    # a = [ra1, dec1]
    # b= [ra2, dec2]
    c1 = SkyCoord(a[0],a[1], unit='deg')
    c2 = SkyCoord(b[0],b[1], unit='deg')
    sep = c1.separation(c2)
    return(sep.degree) 

def coordconvert(ra, dec):
    c = SkyCoord(ra, dec, unit=(u.hourangle, u.deg))
    newra = c.ra.degree
    newdec = c.dec.degree
    return(newra, newdec)

def getwebdacoords(name):
    err_arr = []
    # fetch coordinates
    try:
        # fetch coordinates 
        linkpt1 = "http://www.univie.ac.at/webda/cgi-bin/frame_data_list.cgi?"
        linkpt2 = "+ad2k+ad2000.coo"
        link = linkpt1+name+linkpt2
        page = requests.get(link)
    except:
        #print("Couldn't find coordinate link for", name)
        err_arr.append((name, "Couldn't find {0}".format(link)))
        #print(link)
        return(None, None, None, err_arr)
    try:
        tree = html.fromstring(page.content)
        text = str(etree.tostring(tree))
        temp = text.split(r'\n\n ')
        temp = temp[-1]
        temp = temp.split(r'\n')
        temp = temp[:-2]
        temp = [x.split(r'  ') for x in temp]
        temp = [list(filter(None, x)) for x in temp]
        
        coords = []
        subsetcid = []
        
        for k in np.arange(len(temp)):
            coord = temp[k]
            if '&#' in coord[3]:
                x = coord[3].split(r'&#')
                
                fixeddec = str(x[0])
                coord[3] = fixeddec
            if len(coord[2].split(' ')) < 3:
                coord[2] = '00 '+coord[2]

            try:
                coords.append(coordconvert(coord[2], coord[3]))
                subsetcid.append(coord[0])
            except:
                err_arr.append((name, "coord error: {0}".format(coord)) )
                #print("error with", name, coord)
                continue
        
        if coords == []:
            err_arr = (name,"all coords rejected")
            return(None, None, None, err_arr)

        subsetra = np.asarray([coord[0] for coord in coords])
        subsetdec = np.asarray([coord[1] for coord in coords])
        return(subsetcid, subsetra, subsetdec, err_arr)

    except:
        #print("error with parsing coords html")
        #print(link)
        err_arr.append((name, "html parsing error"))
        return(None, None, None, err_arr)
        
        
        



def getwebdamags(name):
    err_arr = []
    # fetch magnitudes 
    try:
        linkpt1 = "http://www.univie.ac.at/webda/cgi-bin/frame_data_list.cgi?"
        linkpt2 = "+ubvpg+ubv.pgo"
        link = linkpt1+name+linkpt2
        page = requests.get(link)
    except:
        #print("Couldn't find magnitude link for", name)
        #print(link)
        err_arr.append((name, "Couldn't find {0}".format(link)))
        #print(link)
        return(None, None, None, err_arr)
    try:
        tree = html.fromstring(page.content)
        text = str(etree.tostring(tree))
        temp = text.split(r'\n\n ')
        temp = temp[-1]
        temp = temp.split(r'\n')
        temp = temp[:-2]
        temp = [x.split(r'  ') for x in temp]
        temp = [list(filter(None, x)) for x in temp]
        subsetmid =np.asarray([int(x[0]) for x in temp])
        subsetvmag = np.asarray([float(x[2]) for x in temp])
        subsetbv = np.asarray([float(x[3]) for x in temp])
        return(subsetmid, subsetvmag, subsetbv, err_arr)
    except:
        #print("error with parsing mags html")
        #print(link)
        err_arr.append((name, "html parsing error: {0}".format(link)))
        return(None, None, None, err_arr)



In [3]:
%%cython
import numpy as np

def pbsearch(sorted_data, subset, tolerance):
    searched_arr = []
    matched_arr = []
    
    cdef int i, n, N, j, index
    cdef double lower, upper, tol
    N = len(subset)
    for i in range(N):
        a = subset[i]
        # a function of the declination
        tol = abs(tolerance*np.cos(a[1]))
        ra = sorted_data[:]
        n = list.__len__(ra)
        if (i % 2000 == 0):
            print(i, N)
        for j in range(n):
            if (n <= 1):
                break
            index = int(n/2)
            entry = ra[index]
            lower = a[0] - tol
            upper = a[0] + tol
            
            if lower > entry[0]:
                ra = ra[index:]
            elif upper < entry[0]:
                ra = ra[:(index)]
            else:
                # ra match, check dec
                if (abs(a[1] - entry[1]) <= tol):
                    matched_arr.append(entry)
                    searched_arr.append(a)
                    
                ra.pop(index)
                if not ra:
                    break
                
            try:
                n = list.__len__(ra)
            except:
                break
                
    

    print("searched for a subset of ", len(subset), "Coords against 2mass list of", len(sorted_data))
    print("Coord matches: ", len(matched_arr), "with a tolerance of ", tolerance)
    return(searched_arr,matched_arr)

# Load-2Mass-Database

In [156]:

# Read in Jeffrey's 2Mass input database Name, RA, and DEC information
# note this data is presorted by RA
# jsortedgal.csv
df = pd.read_csv('sydunimatches.csv', delimiter=',',index_col=False, header=0, dtype=None)
#df.columns = ["ID", "VMag", "RA", "DEC", "B","V"]
#df.columns = ["ID", "VMag", "RA", "DEC", "B","V"]
'''


df.sort_values("RA")
#all(l[i] <= l[i+1] for i in np.arange(len(l)-1))
# there's some weird sorting problems. A few are multiplied by 10^-4 for some reason
l = (df["RA"])
problems = []
for i in np.arange(len(l)-1):
    if (l[i] <= l[i+1]):
        continue
    else:
        problems.append(i+1)
print(len(problems), "weird sorts")
for problem in problems:
    df["RA"][problem] = df["RA"][problem]*10000

'''

'\n\n\ndf.sort_values("RA")\n#all(l[i] <= l[i+1] for i in np.arange(len(l)-1))\n# there\'s some weird sorting problems. A few are multiplied by 10^-4 for some reason\nl = (df["RA"])\nproblems = []\nfor i in np.arange(len(l)-1):\n    if (l[i] <= l[i+1]):\n        continue\n    else:\n        problems.append(i+1)\nprint(len(problems), "weird sorts")\nfor problem in problems:\n    df["RA"][problem] = df["RA"][problem]*10000\n\n'

/Users/mmcintosh/anaconda/lib/python3.4/site-packages/ipykernel/__main__.py:7: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.


In [5]:
print(len(df))
twomass_ident = np.asarray(df["ID"])
twomass_vmag = np.asarray(df["VMag"])
twomass_ra = np.asarray(df["RA"])
twomass_dec = np.asarray(df["DEC"])
twomass_bv = np.asarray(df["B"] - df["V"])

twomasszip = list(zip(twomass_ra, twomass_dec, twomass_vmag, twomass_bv, twomass_ident))

9073309


In [6]:
# read in webda database
# load objects table
df = pd.read_csv('allwebdacoords.csv', delimiter=',',index_col=False, header=0, dtype=None)
df.columns = ["name","ID","RA","DEC"]
df.head()
webda_name = np.asarray(df["name"])
webda_ident = np.asarray(df["ID"])
webda_ra = np.asarray(df["RA"])
webda_dec = np.asarray(df["DEC"])


webdazip = list(zip(webda_ra, webda_dec, webda_ident, webda_name))

In [10]:
print(webdazip[0], twomasszip[0])

(12.717000000000001, 49.689961111099997, 1, 'ale01') (0.0010089999999999999, -57.713554000000002, 13.429, 0.60699999999999932, 1093660.0)


# Select-Clusters 

In [4]:


#I got this html from querying webda for all clusters with J2000 coordinate data available

# connect to webda database
# grab each cluster name
## search this page: http://www.univie.ac.at/webda/cgi-bin/seldb.cgi?
## for links like http://www.univie.ac.at/webda/cgi-bin/ocl_page.cgi?dirname=wes02
## and grab the string after dirname=  
# these are the clusters that have coordinate data
# do the same for UBV data, and take only the names of clusters that have both
#http://www.univie.ac.at/webda/webda_selection.html
#I queried for UBV photoelectric observations
#and Equatorial coordinates J2000 and saved the resulting table data to acquire the names of the clusters for which this data was available


f = codecs.open("webdacoords.html", "r", "utf-8")
page = f.read()
split = page.split("dirname=")
coordnames = [split[x].split('"')[0] for x in np.arange(len(split))]
coordnames = coordnames[1:]
f = codecs.open("webdamag.html", "r", "utf-8")
page = f.read()
split = page.split("dirname=")
magnames = [split[x].split('"')[0] for x in np.arange(len(split))]
magnames = magnames[1:]

wnindices = [i for (i, x) in enumerate(magnames) if x in coordnames]
webnames = [magnames[i] for i in wnindices]
print(len(webnames))

cnindices = [i for (i, x) in enumerate(magnames) if x in webnames]
isplit = [split[i] for i in cnindices]
temp = [isplit[x].split(";")[1] for x in np.arange(len(isplit))][1:]
clusternames = [temp[x].split("&")[0] for x in np.arange(len(temp))]

# read in dias results
data = genfromtxt('dias_wtol0.005556_matched125.csv', delimiter=',', dtype=None)
data = data[0]
diasnames = [row.decode('UTF-8') for row in data]

fwnindices = [i for (i, x) in enumerate(clusternames) if x in diasnames]
#print([list(clusternames)[i] for i in fwnindices])
filteredwebnames = [(webnames)[i] for i in fwnindices]
#print(filteredwebnames)
#print(len(filteredwebnames))

# it looks like the first elements don't match
names = filteredwebnames[1:]


428


In [ ]:
#allnames = ['bl01', 'bo15', 'ho18', 'ic4665', 'mel066', 'mel071', 'ngc1662', 'ngc2204', 'ngc2215', 'ngc2287', 'ngc2506', 
#         'ngc2516', 'ngc2682', 'ngc6087', 'ngc6208', 'ngc6716',
#         'ic4756', 'ngc2354', 'ngc3680', 'ic4651', 'ngc2112', 'ngc3960', 'ic4665', 'ngc6025']

errnames = ['bl01', 'bo15', 'ho18', 'ic4665', 'mel066', 'mel071', 'ngc1662', 'ngc2204', 'ngc2215', 'ngc2287', 'ngc2506', 'ngc2516', 'ngc2682', 'ngc6087', 'ngc6208', 'ngc6716']

# Webda-Data-Retrieval

In [ ]:

tol = 0.005556
coorddiff = []
y = 15

#['bl01', 'bo15', 'ho18', 'ic4665', 'mel066', 'mel071', 
#'ngc1662', 'ngc2204', 'ngc2215', 'ngc2287', 'ngc2506', 
#'ngc2516', 'ngc2682', 'ngc6087', 'ngc6208', 'ngc6716']
#
#0 has only 1 that is 1.4 away, 
#1 had 1 0.1 away
#4 had 2, 1 and 0.5 away
#5 has 6 but gets some nans
# 6 has 24 and nans
# 10 had 1 that might be a match, but the rest were >2 away
# 12 was really really weird and had nans, also really long, like, 3000
# 13, 14, 15 have everything the same distance away

#for y in tqdm(range(len(names))):
names = errnames
name = names[y]


subsetid, subsetra, subsetdec, err_arr = getwebdacoords(name)
subsetmid, subsetvmag, subsetbv, err_arr = getwebdamags(name)
    
if (subsetid) or (subsetmid) is None:
    print("Retrieval error. Skipping", name)

subsetzip = list(zip(subsetra, subsetdec, subsetvmag, subsetbv, subsetmid,subsetid))

# Database-Search


In [ ]:
tol = 0.005556
print("data retrived, starting pbsearch")
searched_ra, matched_ra = pbsearch(twomasszip, webdazip, tol)

data retrived, starting pbsearch
(0, 1179243)

In [ ]:
print("tested")

tested


In [ ]:
print(matched_ra[1])
print(searched_ra[1])

In [ ]:
# take the euclidean distance between coordinates, magnitude, and color
a = [np.asarray(x[:4]) for x in searched_ra]
b = [np.asarray(x[:4]) for x in matched_ra]

dists = [np.linalg.norm(a[i]-b[i]) for i in np.arange(len(a))]
dists = [dists[~np.isnan(x)] for x in dists]
print(dists)
plt.hist(dists, bins = 100)
plt.show()

# Filtering

In [ ]:
# find the repeat matches, and save those who have the lesser distance
# how do I quantify how sure I am the lesser one is the true one? 
matchedid = []
searchedid = []
d = []
for i in np.arange(len(matched_ra)):
    matchedid.append(matched_ra[i][4])
    searchedid.append(searched_ra[i][4])

tmp = []
for j in np.arange(len(searchedid)):
    x = [i for i, y in enumerate(searchedid) if y == searchedid[j]]
    tmp.append(x)
    
bar = []
for entry in np.unique(tmp):
    try:
        if len(entry) > 1:
            foo = [dists[x] for x in entry]
            #print(entry, foo)
            minindex = [i for i, j in enumerate(foo) if j == min(foo)]
            minentry = entry[minindex[0]]
            #print(minentry, dists[minentry])
            bar.append(minentry)
        else:
            #print(entry, dists[entry[0]])
            bar.append(entry[0])
    except:
        bar.append(entry)


In [ ]:
print(np.unique(tmp))

In [ ]:
# so bar has the closest ones and the non-repeated ones. How far away are these?
closest = np.asarray([dists[x] for x in bar])

# guess a high number of bins
bins = 20#14
data = py.hist(closest, bins = bins)

# is it too small? observed and expected frequencies should be at least 5: http://docs.scipy.org/doc/scipy-0.15.1/reference/generated/scipy.stats.chisquare.html
# grab number in each bin, check that most of them are above 5, otherwise decrease bin number and do it again
thresh = 0.7
toomanybins = len([x for x in data[0] if x >= 5])/len(data[0]) < thresh

while toomanybins:
    bins = bins - 1
    data = py.hist(closest, bins = bins)
    toomanybins = len([x for x in data[0] if x >= 5])/len(data[0]) < thresh
print("using",bins, "bins")
plt.close()

In [ ]:
bins=40

In [ ]:
data = py.hist(closest, bins = bins)
# Generate data from bins as a set of points 
x = [0.5 * (data[1][i] + data[1][i+1]) for i in range(len(data[1])-1)]
y = data[0]

popt, pcov = optimize.curve_fit(gauss, x, y)
x_fit = py.linspace(x[0], x[-1], 100)
y_fit = gauss(x_fit, *popt)


plt.plot(x_fit, y_fit, lw=4, color="r")

cx_fit = py.linspace(x[0], x[-1], len(x))
cy_fit = gauss(cx_fit, *popt)

# check degrees of freedom - number of bins I think
# does this give reduced chi squared? 
# near 1 is a good fit
chi2 = (chisquare(f_obs=y,f_exp=cy_fit))
rchi2 = chi2[0]/(len(x)-3)
print(chi2, rchi2)
plt.xlabel('Angular distance, in degrees')
plt.ylabel('Number of Stars')
plt.title('Euclidean Distances \n mu={0:.2e}, std={1:.2e}, rchi2:{2:.2e}, bins:{3}'.format(popt[1], popt[2], rchi2, bins))
plt.ticklabel_format(style='sci', axis='x', scilimits=(0,0))

# draw 1,2,3 sigma line
for i in [1,2,3]:  
    sx = [popt[1]+popt[2]*i]*(np.max(data[0]))
    sy = np.arange(np.max(data[0]))
    plt.plot(sx,sy, color="r")
plt.show()

In [ ]:
# maybe just take the data w/i 1 sigma of the mean? 

s = 1
mean = popt[1]
std = popt[2]
coordmask = np.asarray([int(dist < (mean+s*std) and dist > (mean-s*std)) for dist in dists])
print("selected", sum(coordmask), "from",len(coordmask))
coordfiltered_searched_ra = np.asarray(searched_ra)[coordmask == 1]
coordfiltered_matched_ra = np.asarray(matched_ra)[coordmask == 1]





In [ ]:
print(coordfiltered_searched_ra)
print(np.where(dists)[0])

In [ ]:
N = sum(coordmask)
if N > 0:

    print(N, "matches for", name, "for tolerance", tol)
    filename = "filteredresults/"+name+"_wtol"+str(tol)+"_matched"+str(N)+'.csv'
    # subsetra, subsetdec, subsetvmag, subsetbv, subsetmid, subsetid
    # twomass_ra, twomass_dec, twomass_vmag, twomass_bv, twomass_ident
    header = ["webdacoordid", "webdaras", "webdadecs", "webdamagid", "webdavmags", "webdabv","galahids", "twomassras", "twomassdecs", "twomassvmags", "twomassbv"]
    with open(filename, 'w', newline='') as fp:
        a = csv.writer(fp, delimiter=',')
        a.writerow(header)
        for i in np.arange(N):
            data = [coordfiltered_searched_ra[i][5], coordfiltered_searched_ra[i][0], coordfiltered_searched_ra[i][1], coordfiltered_searched_ra[i][4], coordfiltered_searched_ra[i][2], coordfiltered_searched_ra[i][3], coordfiltered_matched_ra[i][4], coordfiltered_matched_ra[i][0], coordfiltered_matched_ra[i][1], coordfiltered_matched_ra[i][2], coordfiltered_matched_ra[i][3]]
            a.writerow(data)
else:
    print("No matches for", name, "for tolerance", tol)

# Looping

In [ ]:
# loop this code over a lot of clusters

names = filteredwebnames#coordnames 
tol = 0.005556
coorddiff = []
errnames = []
for y in tqdm(np.arange(len(names))):

    name = names[y]
    print(name)

    subsetid, subsetra, subsetdec, err_arr = getwebdacoords(name)
    subsetmid, subsetvmag, subsetbv, err_arr = getwebdamags(name)
    
    if (subsetid) or (subsetmid) is None:
        print("Retrieval error. Skipping", name)
        continue

    print(name, len(subsetid))
    subsetzip = list(zip(subsetra, subsetdec, subsetvmag, subsetbv, subsetmid,subsetid))
    print("Data retrived, starting pbsearch")
    searched_ra, matched_ra = pbsearch(twomasszip, subsetzip, tol)
    if len(searched_ra) == 0:
        print("No matches. Skipping", name)
        continue

    # take the euclidean distance between coordinates, magnitude, and color
    a = [np.asarray(x[:4]) for x in searched_ra]
    b = [np.asarray(x[:4]) for x in matched_ra]

    dists = [np.linalg.norm(a[i]-b[i]) for i in np.arange(len(a))]
    #plt.hist(dists, bins = 20)
    #plt.show()


    # find the repeat matches, and save those who have the lesser distance
    # how do I quantify how sure I am the lesser one is the true one? 
    try:
        matchedid = []
        searchedid = []
        d = []
        for i in np.arange(len(matched_ra)):
            matchedid.append(matched_ra[i][4])
            searchedid.append(searched_ra[i][4])

        tmp = []
        for j in np.arange(len(searchedid)):
            x = [i for i, y in enumerate(searchedid) if y == searchedid[j]]
            tmp.append(x)

        bar = []
        for entry in np.unique(tmp):
            if len(entry) > 1:
                foo = [dists[x] for x in entry]
                #print(entry, foo)
                minindex = [i for i, j in enumerate(foo) if j == min(foo)]
                minentry = entry[minindex[0]]
                #print(minentry, dists[minentry])
                bar.append(minentry)
            else:
                #print(entry, dists[entry[0]])
                bar.append(entry[0])
    except:
        print(name, "had no repeats")
        bar = np.arange(len(dists))

    try:
        # so bar has the closest ones and the non-repeated ones. How far away are these?
        closest = np.asarray([dists[x] for x in bar])

        # guess a high number of bins
        bins = 20#14
        data = py.hist(closest, bins = bins)

        # is it too small? observed and expected frequencies should be at least 5: http://docs.scipy.org/doc/scipy-0.15.1/reference/generated/scipy.stats.chisquare.html
        # grab number in each bin, check that most of them are above 5, otherwise decrease bin number and do it again
        thresh = 0.7
        toomanybins = len([x for x in data[0] if x >= 5])/len(data[0]) < thresh

        while toomanybins:
            bins = bins - 1
            data = py.hist(closest, bins = bins)
            toomanybins = len([x for x in data[0] if x >= 5])/len(data[0]) < thresh
        if bins < 10:
            bins = 10
        print("using",bins, "bins")
        plt.close()

        data = py.hist(closest, bins = bins)
        # Generate data from bins as a set of points 
        x = [0.5 * (data[1][i] + data[1][i+1]) for i in range(len(data[1])-1)]
        y = data[0]
    except:
        print(name, "had error with histogram")
        errnames.append(name)
        continue

    try:
        popt, pcov = optimize.curve_fit(gauss, x, y)
    except:
        print(name, "had error with optimize.curve_fit")
        errnames.append(name)
        continue
    x_fit = py.linspace(x[0], x[-1], 100)
    y_fit = gauss(x_fit, *popt)


    plt.plot(x_fit, y_fit, lw=4, color="r")

    cx_fit = py.linspace(x[0], x[-1], len(x))
    cy_fit = gauss(cx_fit, *popt)

    # check degrees of freedom - number of bins I think
    # does this give reduced chi squared? 
    # near 1 is a good fit
    chi2 = (chisquare(f_obs=y,f_exp=cy_fit))
    rchi2 = chi2[0]/(len(x)-3)
    print(chi2, rchi2)
    
    
    if not rchi2 or rchi2 > 6:
        print("Error with reduced chi squared:",rchi2, 'for', name)
        errnames.append(name)
        continue
    
    plt.xlabel('Euclidean Distance')
    plt.ylabel('Number of Stars')
    plt.title('Cluster {4} \n mu={0:.2e}, std={1:.2e}, rchi2:{2:.2e}, bins:{3}'.format(popt[1], popt[2], rchi2, bins, name))
    plt.ticklabel_format(style='sci', axis='x', scilimits=(0,0))

    # draw mean line
    sx = [popt[1]]*(np.max(data[0]))
    sy = np.arange(np.max(data[0]))
    plt.plot(sx,sy, color="g")
    
    # draw 1,2,3 sigma line
    for i in [1,2,3]:  
        sx = [popt[1]+popt[2]*i]*(np.max(data[0]))
        sy = np.arange(np.max(data[0]))
        plt.plot(sx,sy, color="r")
    plt.savefig(name+'.png', dpi=300)
    plt.show()

    # maybe just take the data w/i 1 sigma of the mean? 

    s = 1
    coordmask = np.asarray([int(dist < (popt[1]+s*popt[2]) and dist > (popt[1]-s*popt[2])) for dist in dists])
    print("selected", sum(coordmask), "from",len(coordmask))
    coordfiltered_searched_ra = np.asarray(searched_ra)[coordmask == 1]
    coordfiltered_matched_ra = np.asarray(matched_ra)[coordmask == 1]



    N = sum(coordmask)
    if N > 0:

        print(N, "matches for", name, "for tolerance", tol)
        filename = "filteredresults/"+name+"_wtol"+str(tol)+"_matched"+str(N)+'.csv'
        # subsetra, subsetdec, subsetvmag, subsetbv, subsetmid, subsetid
        # twomass_ra, twomass_dec, twomass_vmag, twomass_bv, twomass_ident
        header = ["webdacoordid", "webdaras", "webdadecs", "webdamagid", "webdavmags", "webdabv","galahids", "twomassras", "twomassdecs", "twomassvmags", "twomassbv"]
        with open(filename, 'w', newline='') as fp:
            a = csv.writer(fp, delimiter=',')
            a.writerow(header)
            for i in np.arange(N):
                data = [coordfiltered_searched_ra[i][5], coordfiltered_searched_ra[i][0], coordfiltered_searched_ra[i][1], coordfiltered_searched_ra[i][4], coordfiltered_searched_ra[i][2], coordfiltered_searched_ra[i][3], coordfiltered_matched_ra[i][4], coordfiltered_matched_ra[i][0], coordfiltered_matched_ra[i][1], coordfiltered_matched_ra[i][2], coordfiltered_matched_ra[i][3]]
                a.writerow(data)
    else:
        print("No matches for", name, "for tolerance", tol)
        
print(errnames)

In [ ]:
len(errnames)

In [ ]:
from astropy.io import ascii
import astropy.coordinates as coord
import glob


files = glob.glob("filteredresults/*")

ra_arr = []
dec_arr = []

for file in files:
    data = ascii.read(file, header_start=0, data_start=1)
    ra = coord.Angle(data["twomassras"]*u.degree)
    ra = ra.wrap_at(180*u.degree)
    ra = ra.radian
    ra_arr.extend(ra)

    dec = coord.Angle(data["twomassdecs"]*u.degree)
    dec = dec.radian
    dec_arr.extend(dec)

fig = plt.figure(figsize=(8,6))
ax = fig.add_subplot(111, projection="mollweide")
ax.grid(True)
ax.scatter(ra_arr, dec_arr)
len(files)
ax.set_title("All sky projection of matched GALAH inputs w/ Webda Data \n {0} clusters plotted".format(len(files)))

In [ ]:
# load spectra table

df = pd.read_csv('reduced_matariki_guess_614.csv', delimiter=',',index_col=False, header=0, dtype=float)
df.columns = ["ID", "RA","DEC", "vf", "teff", "logg", "feh", "goodeverything"]
df = df.drop(df[df["goodeverything"] == 0].index)



In [ ]:
# load objects table
df = pd.read_csv('reduced_objects.csv', delimiter=',',index_col=False, header=0, dtype=None)
df.columns = ["type","ra","dec", "mag", "ID"]
df = df.dropna()
df = df.drop(df[df["type"] != 'P'].index)
df.head()


In [ ]:
type(df["ID"][649601])

In [ ]:
from astropy.io import ascii
import glob


files = glob.glob("filteredresults/*")


matched_ids = []
for file in files:
    data = ascii.read(file, header_start=0, data_start=1)
    galahids = data["galahids"]
    for i in np.arange(len(galahids)):
        #print("searching", galahids[i])
        searched = galahids[i]
        found = df[df['ID']==searched].index.tolist()
        if found:
            print("in", file, "matched galah id", searched)
            matched_ids.append(searched)
            
        #print(df[df['ID']==galahids[i]].index.tolist())



In [ ]:
u = np.unique(matched_ids)
print(len(matched_ids), len(u))
print(u)

# plot spec tab ra and dec in red, webda matched ra and dec

In [5]:

names = coordnames
#print(names)
id_arr = []
ra_arr = []
dec_arr = []
name_arr = []


loop = np.arange(len(names))

for y in tqdm(loop):
    name = names[y]
    
    subsetid, subsetra, subsetdec, err_arr = getwebdacoords(name)
    #subsetmid, subsetvmag, subsetbv, err_arr = getwebdamags(name)
    
    if (subsetid) is None:
        print("Retrieval error. Skipping", name)
        #print("errors", err_arr)
        continue
    
    id_arr.extend(subsetid)
    ra_arr.extend(subsetra)
    name_arr.extend([name]*len(subsetid))
    dec_arr.extend(subsetdec)






 27%|██▋       | 260/946 [21:43<16:22,  1.43s/it]

Retrieval error. Skipping bas10
Retrieval error. Skipping

 40%|███▉      | 377/946 [54:22<4:48:51, 30.46s/it]

 cr228
Retrieval error. Skipping

 ic4725


In [8]:
["bas10", "cr228", "ic4725"]

print(len(ra_arr),len(id_arr))

print(len(err_arr))
#print(len(err_arr2))

print(err_arr)
#print(err_arr2)

1179243 1179243
0
[]


In [ ]:
name = names[0]
test = getwebdacoords(name)

In [ ]:
print(len(test))

In [7]:
N = len(id_arr)
# subsetra, subsetdec, subsetvmag, subsetbv, subsetmid, subsetid
# twomass_ra, twomass_dec, twomass_vmag, twomass_bv, twomass_ident
filename = "allwebdacoords.csv"
header = ["name","webdacoordid", "webdaras", "webdadecs"]
with open(filename, 'w', newline='') as fp:
    a = csv.writer(fp, delimiter=',')
    a.writerow(header)
    for j in np.arange(N):
        data = [name_arr[j],id_arr[j], ra_arr[j], dec_arr[j]]
        a.writerow(data)

In [ ]:

tmp = [err[0] for err in err_arr]
newnames = list(set(tmp))

In [ ]:
name = newnames[0]
print(name)

In [ ]:
err_arr = []
# fetch coordinates 
linkpt1 = "http://www.univie.ac.at/webda/cgi-bin/frame_data_list.cgi?"
linkpt2 = "+ad2k+ad2000.coo"
link = linkpt1+name+linkpt2
page = requests.get(link)

tree = html.fromstring(page.content)
text = str(etree.tostring(tree))
temp = text.split(r'\n\n ')
temp = temp[-1]
temp = temp.split(r'\n')
temp = temp[:-2]
temp = [x.split(r'  ') for x in temp]
temp = [list(filter(None, x)) for x in temp]



coords = []
subsetcid = []
for k in np.arange(len(temp)):
    coord = temp[k]
    if '&#' in coord[3]:
        x = coord[3].split(r'&#')
        #y = x[1].split(r';')
        fixeddec = str(x[0])#+str(y[0])
        coord[3] = fixeddec
        

    coords.append(coordconvert(coord[2], coord[3]))
    subsetcid.append(coord[0])

        

if coords == []:
    err_arr2.append((name,"all"))
    print("empty")
    #continue
subsetcid = np.asarray(subsetcid)
subsetra = np.asarray([coord[0] for coord in coords])
subsetdec = np.asarray([coord[1] for coord in coords])

In [ ]:
print(coord)
print(len(coord[2].split(' ')) < 3)

print(coord)

In [ ]:
print(temp)

In [ ]:
id_arr.extend(ids)
ra_arr.extend(subsetra)
name_arr.extend([name]*len(ra_arr))
dec_arr.extend(subsetdec)

In [ ]:
# back up for webda, download files individually
# Read in Webda observed Name, RA, and DEC information
'''

df = pd.read_csv('ad2000.coo', delimiter='\t',index_col=False, header=1)
df.columns = ["ID", "ref", "RA", "DEC"]


coords = [coordconvert(ra, dec) for ra in df["RA"] for dec in df["DEC"]]
subsetra = np.asarray([coord[0] for coord in coords])
subsetdec = np.asarray([coord[1] for coord in coords])
'''

# Read in Gayandhi's 2Mass input database Name, RA, and DEC information
# note this data is presorted by RA
'''

df = pd.read_csv('gsortedgal.csv', delimiter=',',index_col=False, header=0)
df.columns = ["ID", "RA", "DEC"]

# get rid of blank entries
print(len(df))
df = df.loc[(df != 0).any(1)]
print(len(df))
print(df.head())

# assign to np arrays
twomass_ident = np.asarray(df["ID"])
twomass_ra = np.asarray(df["RA"])
twomass_dec = np.asarray(df["DEC"])

'''

# dias data
'''

# Read in DIAS 
# I didn't know an easy way to read in this file so this is very messy and hacked and slow and I apologize


dias_data = np.genfromtxt('dias.txt', delimiter='\t', names=None, dtype=None)
dias_data = dias_data[1:]

data = []
subsetid = []
subsetra = []
subsetdec = []
for line in dias_data:
 
    line = line.decode('UTF-8')

    line = (str(line).split("  "))

    line = np.array(list(filter(None, line)))
    subsetid.append(line[0])
    try:
        c = SkyCoord(line[1], line[2], unit=(u.hourangle, u.deg))
        subsetra.append(c.ra.degree)
        subsetdec.append(c.dec.degree)
    except:
        print("error: ", line)
    data.append(line)
    
# some problems with dec
subsetra = np.asarray(subsetra)
subsetdec = np.asarray(subsetdec)
name = "dias"


'''

[Back to Top](#Table-of-Contents) 


In [ ]:
# stuff for SYD UNI

df = pd.read_csv('sydunimatches.csv', delimiter=',',index_col=False, header=0, dtype=None)
names = list(set(df["name"]))

i = 0
for name in names: 
    subsetmid, subsetvmag, subsetbv, err_arr = getwebdamags(name)
    if subsetvmag == None:
        continue
    magdf = pd.DataFrame({'name':([name]*len(subsetmid)), 'webdacoordid': subsetmid, 'vmag':subsetvmag, 'bv':subsetbv})
    # add on to df
    
    if i < 1:
        df = pd.merge(df, magdf, on=['name', 'webdacoordid'], how='left')
    else:
        df.update(magdf)
    i+=1
    break

# filter by magnitude and drop duplicate matches
df = df.drop_duplicates(subset=["name", "webdacoordid"])
df = df.drop_duplicates(subset=["webdaras", "webdadecs"])
filtered = df[(df["vmag"] <= 14.5) & (magdf["vmag"] >= 9) & pd.notnull(df["ra"])]

df.rename(columns={'webdacoordid':'starID'}, inplace=True)

header = ["webdaras", "webdadecs", "name", "starID", "vmag"]
df.to_csv('mfsyduni_all.csv', columns = header, index=False)
mdf = df[df["vmag"].notnull()]
mdf.to_csv('mfsyduni_wmag.csv', columns = header, index=False)